In [1]:
#output csv
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import time

def createDataFrame1(t,u):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    url_list = ["https://tfc-taiwan.org.tw" + url.get('href') for url in u]

    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list, 'url': url_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list, 'url': url_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    df['tag'] = np.where(cond1, '__label__fake', '__label__undefined')
    df = df[['tag','url','title']]

def createDataFrame2(t,u):
    #put data into a list
    title_list = [p.text.strip() for p in t]
    url_list = [url.get('href') for url in u]

    global df
    #check if dataframe exist
    if c == 0:
        df = pd.DataFrame({'title': title_list, 'url': url_list})
    else:
        df = df.append(pd.DataFrame({'title': title_list, 'url': url_list}), ignore_index = True)
    
    #check fake
    cond1 = df['title'].str.contains('【錯誤】')
    df['tag'] = np.where(cond1, '__label__fake', '__label__undefined')
    df = df[['tag','url','title']]

def outputCsv(): 
    #input news article URL
    #[早安健康NEWS 醫療新聞]undefined https://news.everydayhealth.com.tw/category/news
    #[早安健康NEWS 疾病知識]undefined https://news.everydayhealth.com.tw/category/knowledge
    #fake https://tfc-taiwan.org.tw/articles/category/26/27

    fake_URL = "https://tfc-taiwan.org.tw/taxonomy/term/475"
    undefined_URL = "https://news.everydayhealth.com.tw/category/knowledge"
    global c
    c = 0
    
    for i in range(11):
        req = request.Request(fake_URL)
        req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
        response = request.urlopen(req)
        data = response.read().decode("utf-8")
        
        #read data in html format
        html = BeautifulSoup(data,"html.parser")
        
        #set find area
        t = html.find_all("h3", class_= "article-title")
        u = html.select("h3.article-title a")

        #create dataframe
        createDataFrame1(t,u) 
        
        next_link = html.find("a", string = "下一頁 ›")
        next_url = "https://tfc-taiwan.org.tw/taxonomy/term/475" + next_link["href"]
        fake_URL = next_url
        
        c+=1
        time.sleep(3)
    
    for i in range(20):
        req = request.Request(undefined_URL)
        req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
        response = request.urlopen(req)
        data = response.read().decode("utf-8")
        
        #read data in html format
        html = BeautifulSoup(data,"html.parser")
        
        #set find area
        t = html.find_all("div", id= "essay-title")
        u = html.select("h1 a")

        #create dataframe
        createDataFrame2(t,u) 
        
        next_link = html.find("a", string = "下一頁›")
        next_url = "https://news.everydayhealth.com.tw/category/knowledge" + next_link["href"]
        undefined_URL = next_url
        
        time.sleep(3)
    
    df['title'] = df['title'].str.replace('【錯誤】','').str.replace('【部分錯誤】','').str.replace('網傳','')
    pd.set_option('max_colwidth',100)    
    display(df.head())

    #output csv file
    path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
    df.to_csv(path + 'source.csv', encoding='utf_8_sig', index=False, header=False)

if __name__ == '__main__':
    outputCsv()


,tag,url,title
0,__label__fake,https://tfc-taiwan.org.tw/articles/4025,「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？
1,__label__fake,https://tfc-taiwan.org.tw/articles/4016,"內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？"
2,__label__undefined,https://tfc-taiwan.org.tw/articles/3975,「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？
3,__label__fake,https://tfc-taiwan.org.tw/articles/3968,「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？
4,__label__fake,https://tfc-taiwan.org.tw/articles/3953,「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？


In [2]:
global content_list
content_list = []

def addC(t): 
    temp = []
    for p in t:
        temp.append(p.text.strip())
    content_list.append(temp)

def getContent(url):
    req = request.Request(url)
    req.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36")
    response = request.urlopen(req)
    data = response.read().decode("utf-8")

    #read data in html format
    html = BeautifulSoup(data,"html.parser")

    #set find area
    t = html.find_all("p")
    
    #add new content into list
    addC(t)
          
    time.sleep(3) 
    
for url in df['url']:
    getContent(url)
    #print(url)    

print(content_list)

人的困擾可以迎刃而解。', '根據1996年美國國家衛生院調查，美國有三百萬人常患便秘。大部分是女性及65歲以上的成年人，懷孕的婦女在生產或手術後便秘是常見的問題。 便秘也是美國人腸胃問題中最常見的，每年約有兩百萬人次的就診數。然而大多數的民眾均自行買葯解決，從美國人每年花費在緩瀉劑的金錢之多便可窺之一二。', '要了解便秘，須先了解大腸的功能。當食物進入大腸，它一邊吸收水分一邊製造排泄物，即糞便。大腸的肌肉收縮把糞便推向直腸，當糞便到達直腸時，由於大部分的水分已被吸收了，因此糞便變硬。當大腸吸收了太多的水分或者大腸的蠕動變慢時，糞便因通過大腸太慢而變得又乾又硬。', '1.食物中纖維質不足：如老年人因牙齒不好，只吃軟食而導致便秘。', '2.水分不足：每天至少應有八大杯的水分，含咖啡因的飲料如咖啡、可樂及酒精類均有脫水的反作用。', '3.缺乏活動或運動', '4.服用藥物造成：', '如止痛劑、含鋁及鈣質的制酸劑、鈣離子阻斷劑之降血壓藥、抗巴金森氏藥物、抗痙攣藥物、抗憂鬱藥物、鐵劑、利尿劑、抗抽筋藥物等。', '5.大腸激躁症候群：常常與壓力有關。', '6.環境變化因素：如懷孕、老年、旅行等。', '7.不當使用緩瀉劑：', '緩瀉劑會傷害大腸內神經細胞，使其失去正常的收縮能力。', '8.忽略排便的意圖：將逐漸失去便意。', '9.罹患某些疾病：', '神經性（如中風、巴金森氏症、脊髓損傷），代謝性及內分泌疾病﹝如糖尿病、甲狀腺亢進或低下症、尿毒﹞，紅斑性狼瘡等等。', '10.大腸直腸的問題：腸阻塞、腸沾黏等。', '11.腸機能障礙（如：慢性不明原因的便秘）。', '大多數的病人只需要改變飲食及多加運動即可改善便秘情況，醫師會視病人便秘的嚴重性，年齡，以及是否糞便帶血，近期大便習慣是否變化或體重是否減輕而決定須做哪些檢查。通常肛門指診檢查評估肛門括約肌的鬆緊度，壓痛、阻塞或帶血。有些病人要檢查甲狀腺疾病或血中鈣濃度。症狀嚴重的病人需檢查大腸直腸蠕動速度以及直腸肛門機能試驗。在年紀大的病人有時應加作鋇劑灌腸檢查，以及直腸鏡或大腸鏡檢查。', '1.飲食改變', '2.生活方式變化', '3.短時間內使用緩瀉劑： 緩瀉劑有數種', '糞便體積增加的緩瀉劑：如 Metamucil, Citrucel, Konsyl, and Serutan.', '大腸刺激劑

In [3]:
#convert list to dataframe
df['content'] = content_list

print(df.head())

#output csv file
path = "C:\\Users\\user\Desktop\\newscrawler\\outputcsv\\csvdata\\"
df.to_csv(path + 'data3.csv', encoding='utf_8_sig', index=False, header=False)

tag                                      url  \
0       __label__fake  https://tfc-taiwan.org.tw/articles/4025   
1       __label__fake  https://tfc-taiwan.org.tw/articles/4016   
2  __label__undefined  https://tfc-taiwan.org.tw/articles/3975   
3       __label__fake  https://tfc-taiwan.org.tw/articles/3968   
4       __label__fake  https://tfc-taiwan.org.tw/articles/3953   

                                                                  title  \
0                              「洋蔥不能和魚、蝦、海帶、蜂蜜一起吃...會消化不良、產生結石、導致眼睛失明」？   
1  內容農場文章宣稱「大藥廠打死也不願說的秘密 原來抗癌可以那麼便宜...檸檬是一個神奇的植物，可殺死癌細胞。功效10,000倍強於化療」？   
2                       「家中長輩失智症是腦部血流堵塞造成。多吃藍莓，12天讓腦袋活過來，可抗腫瘤，促進腦部血流量」？   
3          「陳惠仁教授強調...熱苦瓜～只殺癌細胞！切２～３薄苦瓜片放在杯子裡， 加入熱水，會變鹼性水...苦瓜汁能調整高血壓」？   
4        「楊梅是皮蛇（帶狀皰疹）的尅星...10分鐘驗完血後，女醫生確定是得了帶狀皰疹...吃了兩天的楊梅後，居然神經不痛康復了」？   

                                                                                               content  
0  [Taiwan FactCheck Center, 經查：, 一、洋蔥為低草酸食物，正常攝取，不會有「洋蔥與魚同吃，造成人體